In [1]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [4]:
sample_text = """"
Data governance is everything you do to ensure data is secure, private, accurate, available, and usable. It includes the actions people must take, the processes they must follow, and the technology that supports them throughout the data life cycle.

Data governance means setting internal standards—data policies—that apply to how data is gathered, stored, processed, and disposed of. It governs who can access what kinds of data and what kinds of data are under governance. Data governance also involves complying with external standards set by industry associations, government agencies, and other stakeholders. 

Make better, more timely decisions
Users throughout your organization get the data they need to reach and service customers, design and improve products and services, and seize opportunities for new revenues.

Improve cost controls
Data helps you manage resources more effectively. Because you can eliminate data duplication caused by information silos, you don't overbuy—and have to maintain—expensive hardware.

Enhance regulatory compliance
An increasingly complex regulatory climate has made it even more important for organizations to establish robust data governance practices. You avoid risks associated with noncompliance while proactively anticipating new regulations.

Manage risk more easily
With strong governance, you can allay concerns about exposure of sensitive data to individuals or systems who lack proper authorization, security breaches from malicious outsiders, or even insiders accessing data they don’t have the right to see. 

"""

In [7]:
template ="""
Vous êtes traducteur assermenté et vous êtes chargé de traduire le document suivant de l'anglais en français. La traduction que vous rédigez doit être surtout claire et correcte et il devrait surtout préserver le sens du texte original. Le langage que vous utilisez est naturel et compréhensible. D'ailleurs, vous vous permettez d'utiliser des expressions idiomatiques où vous en avez besoin.

Document:
{document}
"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(temperature=0)

In [14]:
translation_chain = ({"document": RunnablePassthrough()}
         | prompt
         | model
         | StrOutputParser()
)

In [10]:
output = translation_chain.invoke(sample_text)

In [12]:
print(output)

La gouvernance des données consiste à tout ce que vous faites pour garantir que les données sont sécurisées, privées, précises, disponibles et utilisables. Cela inclut les actions que les gens doivent prendre, les processus qu'ils doivent suivre et la technologie qui les soutient tout au long du cycle de vie des données.

La gouvernance des données signifie établir des normes internes - des politiques de données - qui s'appliquent à la manière dont les données sont collectées, stockées, traitées et éliminées. Elle régit qui peut accéder à quel type de données et quels types de données relèvent de la gouvernance. La gouvernance des données implique également de se conformer aux normes externes établies par des associations industrielles, des agences gouvernementales et d'autres parties prenantes.

Prendre de meilleures décisions, plus rapidement
Les utilisateurs de toute votre organisation obtiennent les données dont ils ont besoin pour atteindre et servir les clients, concevoir et amél

In [24]:
critique_template = """"
Vous êtes traducteur professionnel et vous révisez la traduction d'un collègue. Veuillez lire attentivement le document original et sa traduction et donner vos commentaires pour que le traducteur puisse améliorer sa traduction. Vos commentaires pourraient prendre en compte la fluidité de langage et la préservation du sens original du texte. Le document que vous traduisez est destiné à une audience Canadien.

Document originale en anglais :
{document}

Traduction :
{translation}
"""

critique_prompt = ChatPromptTemplate.from_template(critique_template)

In [40]:
critique_chain = {"document": RunnablePassthrough(), "translation": RunnablePassthrough()} | critique_prompt | model | StrOutputParser()

In [47]:
rewrite_template = """"
Vous êtes traducteur et vous venez de recevoir des commentaires d'un collègue concernant l'une de vos traductions. Veuillez réviser la traduction en prenant en compte ses commentaires.

Document originale en anglais :
{document}

Traduction :
{translation}

Commentaires :
{critique}

Révision:
"""

rewrite_prompt = ChatPromptTemplate.from_template(rewrite_template)

In [48]:
rewrite_translation_chain = (
    {
        "document": RunnablePassthrough(),
        "translation": RunnablePassthrough() | translation_chain,
    }
    | {
        "document": RunnablePassthrough(),
        "translation": RunnablePassthrough(),
        "critique": RunnablePassthrough() | critique_chain,
    }
    | rewrite_prompt
    | model
    | StrOutputParser()
)

In [49]:
translation = rewrite_translation_chain.invoke(sample_text)

In [50]:
print(translation)

La gouvernance des données est tout ce que vous faites pour garantir que les données sont sécurisées, privées, précises, disponibles et utilisables. Cela inclut les actions que les personnes doivent prendre, les processus qu'elles doivent suivre et la technologie qui les soutient tout au long du cycle de vie des données.

La gouvernance des données signifie établir des normes internes - des politiques de données - qui s'appliquent à la manière dont les données sont collectées, stockées, traitées et éliminées. Elle régit qui peut accéder à quels types de données et quels types de données relèvent de la gouvernance. La gouvernance des données implique également de se conformer aux normes externes établies par des associations industrielles, des agences gouvernementales et d'autres parties prenantes.

Prendre de meilleures décisions, plus rapidement
Les utilisateurs de votre organisation obtiennent les données dont ils ont besoin pour atteindre et servir les clients, concevoir et améliore